# Maximum a posteriori (MAP)

- Hasta ahora nos hemos enfocado en la verosimilitud $p(\mathcal{D}|\theta)$
- En realidad lo que más nos interesa es el *posterior* $p(\theta|\mathcal{D})$

Bayes nos dice que

$$
p(\theta|\mathcal{D}) = \frac{p(\mathcal{D}|\theta)p(\theta)}{p(\mathcal{D})}
$$

El criterio MAP busca encontrar $\theta$ que maximiza el posterior. 

Es decir la moda del posterior

Usando el logaritmo para escribir

$$
\begin{align}
\hat \theta &= \text{arg}\max_\theta \log p(\theta|\mathcal{D}) \nonumber \\
&= \text{arg}\max_\theta \log p(\mathcal{D}|\theta) + \log p(\theta) \nonumber
\end{align}
$$
donde omitimos la evidencia $p(\mathcal{D})$ ya que no depende de $\theta$

- El criterio MAP consiste en maximizar la **verosimilitud** más el **prior**
- Usamos el **prior** para agregar información sobre $\theta$


### Ejemplo: Regresión  con MAP

Quiero modelar una regresión de $N$ parámetros
$$
\begin{align}
y_i = & \Phi(x_i) \theta + \varepsilon_i \nonumber \\
&\varepsilon_i \sim \mathcal{N}(0, \sigma_\epsilon^2) \nonumber \\
&y_i|\theta \sim \mathcal{N}(\Phi(x_i) \theta, \sigma_\epsilon^2) \nonumber
\end{align}
$$
es decir **verosimilitud** Gaussiana tal como antes

Adicionalmente asumiremos que $p(\theta) = \mathcal{N}(0, \sigma_\theta^2)$, es decir **prior** Gaussiano

El estimador MAP es 

$$
\begin{align}
\hat \theta &= \text{arg} \max_\theta -\frac{1}{2} \log(2\pi\sigma_\varepsilon^2) M - \frac{1}{2\sigma_\varepsilon^2} \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2 -\frac{1}{2} \log(2\pi \sigma_\theta^2) N - \frac{1}{2\sigma_\theta^2} \sum_{j=1}^N \theta_j^2 \nonumber \\
&= \text{arg} \min_\theta  \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2 + \frac{\sigma_\varepsilon^2}{\sigma_\theta^2} \sum_{j=1}^N \theta_j^2 \nonumber 
\end{align}
$$


#### Reflexión
- Si hay mucho ruido ($\sigma_\varepsilon^2 \to \infty$) entonces ignoro la verosimilitud y le creo al prior
- Si uso un prior que no me da información ($\sigma_\theta^2 \to \infty$) entonces lo ignoro y me enfoco en la verosimilitud

#### Relación con lo visto sobre regularización

- Si decimos $\lambda = \frac{\sigma_\varepsilon^2}{\sigma_\theta^2}$ entonces la formulación es equivalente a Mínimos cuadrados regularizado, Ridge Regressión, Tikhonov!
- Si hubieramos asumido una distribución Laplaciana para $p(\theta)$ hubieramos obtenido el [LASSO](https://en.wikipedia.org/wiki/Lasso_(statistics))


# Inferencia Bayesiana

- Asumimos que $\theta$ es una variable aleatoria y que tiene cierta distribución
- Buscamos estimar el posterior completo $p(\theta|\mathcal{D})$
- El problema es que $p(\mathcal{D})$ es generalmente incalculable
- Para proseguir tenemos tres opciones
    1. Usar distribuciones que sean conjugadas
    1. Usar una aproximación (inferencia variacional)
    1. Aprender un sistema que se comporte como el posterior (Markov Chain Monte Carlo) 

In [ ]:
import pymc3 as pm
print('Running on PyMC3 v{}'.format(pm.__version__))

temperatura, consumo = df["temperatura"].values, df["consumo"].values
with pm.Model() as helados:
    sigma = pm.HalfNormal('sigma', sd=10, shape=1)
    theta0 = pm.Normal('intercept', mu=0, sd=10, shape=1)
    theta1 = pm.Normal('slope', mu=0, sd=10, shape=1)
    mu = temperatura*theta1 + theta0
    x_observed = pm.Normal('x_obs', mu=mu, sd=sigma, observed=consumo)
    trace = pm.sample(draws=500, tune=200, init='advi', n_init=30000, 
                      cores=4, chains=2, live_plot=False)
display(pm.summary(trace))
pm.traceplot(trace, figsize=(9, 6), combined=True);

Más sobre estos temas en el [repositorio de INFO337](https://github.com/magister-informatica-uach/INFO337/blob/master/MCMC/lecture.ipynb) del magíster de informática y en [Ipython](https://ipython-books.github.io/73-getting-started-with-bayesian-methods/) [cookbook](https://ipython-books.github.io/77-fitting-a-bayesian-model-by-sampling-from-a-posterior-distribution-with-a-markov-chain-monte-carlo-method/)

## FUTURO

Más sobre métodos de monte-carlo
- https://towardsdatascience.com/a-zero-math-introduction-to-markov-chain-monte-carlo-methods-dcba889e0c50
- https://skymind.ai/wiki/markov-chain-monte-carlo
- https://help.xlstat.com/customer/en/portal/articles/2062457-which-statistical-test-should-you-use-